In [150]:
import numpy as np
from keras.models import Model, Sequential
from keras.layers import Input, Add, Convolution2D, ZeroPadding2D,Cropping2D, MaxPooling2D, Flatten, Dense, Dropout, Activation, BatchNormalization, ReLU, Softmax, Conv2DTranspose, Concatenate
#from tensorflow.keras.utils import load_img, save_img, img_to_array
from keras.applications.imagenet_utils import preprocess_input
from keras.preprocessing import image

In [2]:
import keras
keras.__version__

'2.5.0'

In [151]:
from scipy.io import loadmat

In [152]:
dataP = loadmat('../shen/net_P_P_S_F.mat', matlab_compatible=False, struct_as_record=False)
dataG = loadmat('../shen/net_G_P_S_F.mat', matlab_compatible=False, struct_as_record=False)

In [153]:
netP = dataP['net_P']
netG = dataG['net_G']

In [154]:
netG[0][17].w.shape

(11, 11, 17, 64)

In [155]:
netG = dataG['net_G']

In [156]:
netG[0][0].__dict__

{'_fieldnames': ['w', 'b'],
 'w': array([[[[ 2.30315812e-02,  1.81509405e-02, -1.74834970e-02, ...,
           -2.23121326e-03, -1.30789112e-02, -1.56280156e-02],
          [ 2.34974734e-02,  9.14834626e-03, -1.56884808e-02, ...,
           -1.69955380e-02, -4.78502922e-03, -2.37178858e-02],
          [ 1.91959664e-02,  2.22276710e-02, -1.73494946e-02, ...,
           -1.53445499e-02,  1.56322354e-03, -1.16360979e-02],
          ...,
          [-1.96070108e-03,  9.22270212e-03,  5.85072627e-03, ...,
           -6.12970721e-03,  1.04638806e-03,  1.02004530e-02],
          [ 1.78295150e-02, -5.16163418e-04, -9.60542995e-04, ...,
            1.15015823e-02, -3.29929078e-03,  3.87474941e-03],
          [-2.59593921e-03,  4.28012060e-03,  5.22051426e-03, ...,
           -5.45246014e-03, -6.71258802e-03,  2.97084800e-03]],
 
         [[ 1.78936608e-02,  9.10111237e-03, -1.28053660e-02, ...,
            5.20341913e-04, -1.39603801e-02, -6.14321698e-03],
          [ 1.61106698e-02, -4.34159748

In [157]:
net = np.concatenate((netP,netG), axis = 1)
net.shape

(1, 45)

In [158]:
# https://towardsdatascience.com/building-a-resnet-in-keras-e8f1322a49ba

In [159]:
#modelP = Sequential()

# net_P

inputImg = Input(shape=(128,128,3))
inputImg0 = Input(shape=(64,64,3))
zeroPad_1 = ZeroPadding2D((1,1), name= "P_zero_1")(inputImg)
convImg_1 = Convolution2D(16, (3, 3), strides=(1,1), name= "P_conv_1")(zeroPad_1)
batchImg_1 = BatchNormalization(epsilon=1.0000e-04, name= "P_batch_1")(convImg_1)
reluImg_1 = ReLU(name= "P_relu_1")(batchImg_1)
poolImg_1 = MaxPooling2D(pool_size=(2,2), strides=(2,2), name= "P_maxpool_1")(reluImg_1)

zeroPad_2 = ZeroPadding2D((1,1), name= "P_zero_2")(poolImg_1)
convImg_2 = Convolution2D(32, (3, 3), strides=(1,1), name= "P_conv_2")(zeroPad_2)
batchImg_2 = BatchNormalization(epsilon=1.0000e-04, name= "P_batch_2")(convImg_2)
reluImg_2 = ReLU(name= "P_relu_2")(batchImg_2)
poolImg_2 = MaxPooling2D(pool_size=(2,2), strides=(2,2), name= "P_maxpool_2")(reluImg_2)

zeroPad_3 = ZeroPadding2D((2,2), name= "P_zero_3")(poolImg_2)
convImg_3 = Convolution2D(64, (5, 5), strides=(1,1), name= "P_conv_3")(zeroPad_3)
batchImg_3 = BatchNormalization(epsilon=1.0000e-04, name= "P_batch_3")(convImg_3)
reluImg_3 = ReLU(name= "P_relu_3")(batchImg_3)
poolImg_3 = MaxPooling2D(pool_size=(2,2), strides=(2,2), name= "P_maxpool_3")(reluImg_3)

zeroPad_4 = ZeroPadding2D((1,1), name= "P_zero_4")(poolImg_3)
convImg_4 = Convolution2D(128, (3, 3), strides=(1,1), name= "P_conv_4")(zeroPad_4)
batchImg_4 = BatchNormalization(epsilon=1.0000e-04, name= "P_batch_4")(convImg_4)
reluImg_4 = ReLU(name= "P_relu_4")(batchImg_4)
poolImg_4 = MaxPooling2D(pool_size=(2,2), strides=(2,2), name= "P_maxpool_4")(reluImg_4)

zeroPad_5 = ZeroPadding2D((1,1), name= "P_zero_5")(poolImg_4)
convImg_5  = Convolution2D(256, (3, 3), strides=(1,1), name= "P_conv_5")(zeroPad_5)
batchImg_5 = BatchNormalization(epsilon=1.0000e-04, name= "P_batch_5")(convImg_5)
reluImg_5  = ReLU(name= "P_relu_5")(batchImg_5)
poolImg_5 = MaxPooling2D(pool_size=(2,2), strides = (2,2), name= "P_maxpool_5")(reluImg_5)

zeroPad_6 = ZeroPadding2D((1,1), name= "P_zero_6")(poolImg_5)
convImg_6  = Convolution2D(512, (3, 3), strides=(1,1), name= "P_conv_6")(zeroPad_6)
batchImg_6 = BatchNormalization(epsilon=1.0000e-04, name= "P_batch_6")(convImg_6)
reluImg_6  = ReLU(name= "P_relu_6")(batchImg_6)

# upsample
de_deconv_1 = Conv2DTranspose(512, (4, 4), strides=(2,2), dilation_rate=(1,1), name= "P_de_deconv_7")(reluImg_6)
de_crop_1 = Cropping2D(cropping=((1, 1), (1, 1)))(de_deconv_1)
de_zeroPad_1 = ZeroPadding2D((1,1), name= "P_de_zero_7")(de_crop_1)
de_convImg_1 = Convolution2D(256, (3, 3), strides=(1,1), name= "P_de_conv_7")(de_zeroPad_1)
de_batchImg_1 = BatchNormalization(epsilon=1.0000e-04, name= "P_de_batch_7")(de_convImg_1)
## bw'lere hiç bakmadan ekledik. ne zaman halledicez?
de_reluImg_1 = ReLU(name= "P_de_relu_7")(de_batchImg_1)
de_sumImg_1 = Add(name= "add_1")([de_reluImg_1, reluImg_5])


de_deconv_2 = Conv2DTranspose(256, (4, 4), strides=(2,2), dilation_rate=(1,1), name= "P_de_deconv_8")(de_sumImg_1)
de_crop_2 = Cropping2D(cropping=((1, 1), (1, 1)))(de_deconv_2)
de_zeroPad_2 = ZeroPadding2D((1,1), name= "P_de_zero_8")(de_crop_2)
de_convImg_2 = Convolution2D(128, (3, 3), strides=(1,1), name= "P_de_conv_8")(de_zeroPad_2)
de_batchImg_2 = BatchNormalization(epsilon=1.0000e-04, name= "P_de_batch_8")(de_convImg_2)
de_reluImg_2 = ReLU(name= "P_de_relu_8")(de_batchImg_2)
de_sumImg_2 = Add(name = "add_2")([de_reluImg_2, reluImg_4])

de_deconv_3 = Conv2DTranspose(128, (4, 4), strides=(2,2), dilation_rate=(1,1), name= "P_de_deconv_9")(de_sumImg_2)
de_crop_3 = Cropping2D(cropping=((1, 1), (1, 1)))(de_deconv_3)
de_zeroPad_3 = ZeroPadding2D((1,1), name= "P_de_zero_9")(de_crop_3)
de_convImg_3 = Convolution2D(64, (3, 3), strides=(1,1), name= "P_de_conv_9")(de_zeroPad_3)
de_batchImg_3 = BatchNormalization(epsilon=1.0000e-04, name= "P_de_batch_9")(de_convImg_3)
de_reluImg_3 = ReLU(name= "P_de_relu_9")(de_batchImg_3)
de_sumImg_3 = Add(name = "add_3")([de_reluImg_3, reluImg_3])

de_deconv_4 = Conv2DTranspose(64, (4, 4), strides=(2,2), dilation_rate=(1,1), name= "P_de_deconv_10")(de_sumImg_3)
de_crop_4 = Cropping2D(cropping=((1, 1), (1, 1)))(de_deconv_4)
de_zeroPad_4 = ZeroPadding2D((1,1), name= "P_de_zero_10")(de_crop_4)
de_convImg_4 = Convolution2D(32, (3, 3), strides=(1,1), name= "P_de_conv_10")(de_zeroPad_4)
de_batchImg_4 = BatchNormalization(epsilon=1.0000e-04, name= "P_de_batch_10")(de_convImg_4)
de_reluImg_4 = ReLU(name= "P_de_relu_10")(de_batchImg_4)
de_sumImg_4 = Add(name = "add_4")([de_reluImg_4, reluImg_2])

de_deconv_5 = Conv2DTranspose(32, (4, 4), strides=(2,2), dilation_rate=(1,1), name= "P_de_deconv_11")(de_sumImg_4)
de_crop_5 = Cropping2D(cropping=((1, 1), (1, 1)))(de_deconv_5)
de_zeroPad_5 = ZeroPadding2D((1,1), name= "P_de_zero_11")(de_crop_5)
de_convImg_5 = Convolution2D(16, (3, 3), strides=(1,1), name= "P_de_conv_11")(de_zeroPad_5)
de_batchImg_5 = BatchNormalization(epsilon=1.0000e-04, name= "P_de_batch_11")(de_convImg_5)
de_reluImg_5 = ReLU(name= "P_de_relu_11")(de_batchImg_5)

de_zeroPad_6 = ZeroPadding2D((1,1), name= "P_de_zero_12")(de_reluImg_5)
de_convImg_parsing = Convolution2D(11, (3, 3), strides=(1,1), name= "P_de_conv_12")(de_zeroPad_6)


# net_G
modelG_Input = Input(shape=(128,128,11))
G_softImg_1 = Softmax(name= "G_soft_1")(modelG_Input)
G_poolImg_1 = MaxPooling2D(pool_size=(2,2), strides = (2,2), name= "G_maxpool_1")(G_softImg_1)
G_concatImg_1 = Concatenate(name= "G_concat_1")([inputImg0, G_poolImg_1])


G_zeroPad_1 = ZeroPadding2D((5,5), name= "G_zero_1")(G_concatImg_1)
G_convImg_1 = Convolution2D(64, (11, 11), strides=(1,1), name= "G_conv_1")(G_zeroPad_1)
G_reluImg_1 = ReLU(name= "G_relu_1")(G_convImg_1)

G_zeroPad_2 = ZeroPadding2D((2,2), name= "G_zero_2")(G_reluImg_1)
G_convImg_2 = Convolution2D(64, (5, 5), strides=(1,1), name= "G_conv_2")(G_zeroPad_2)
G_reluImg_2 = ReLU(name= "G_relu_2")(G_convImg_2)

G_zeroPad_3 = ZeroPadding2D((2,2), name= "G_zero_3")(G_reluImg_2)
G_convImg_3 = Convolution2D(64, (5, 5), strides=(1,1), name= "G_conv_3")(G_zeroPad_3)
G_reluImg_3 = ReLU(name= "G_relu_3")(G_convImg_3)

G_res1_zeroPad_1_1 = ZeroPadding2D((2,2), name= "G_res1_zero_1_1")(G_reluImg_3)
G_res1_convImg_1_1 = Convolution2D(64, (5, 5), strides=(1,1), name= "G_res1_conv_1_1")(G_res1_zeroPad_1_1)
G_res1_reluImg_1_1 = ReLU(name= "G_res1_relu_1_1")(G_res1_convImg_1_1)
G_res1_zeroPad_1_2 = ZeroPadding2D((2,2), name= "G_res1_zero_1_2")(G_res1_reluImg_1_1)
G_res1_convImg_1_2 = Convolution2D(64, (5, 5), strides=(1,1), name= "G_res1_conv_1_2")(G_res1_zeroPad_1_2) #### !!!!!!!!!!!11
G_res1_sumImg_1 = Add(name = "add_5")([G_res1_convImg_1_2, G_reluImg_3])
G_res1_reluImg_1_2 = ReLU(name= "G_res1_relu_1_2")(G_res1_sumImg_1)

G_res1_zeroPad_2_1 = ZeroPadding2D((2,2), name= "G_res1_zero_2_1")(G_res1_reluImg_1_2)
G_res1_convImg_2_1 = Convolution2D(64, (5, 5), strides=(1,1), name= "G_res1_conv_2_1")(G_res1_zeroPad_2_1)
G_res1_reluImg_2_1 = ReLU(name= "G_res1_relu_2_1")(G_res1_convImg_2_1)
G_res1_zeroPad_2_2 = ZeroPadding2D((2,2), name= "G_res1_zero_2_2")(G_res1_reluImg_2_1)
G_res1_convImg_2_2 = Convolution2D(64, (5, 5), strides=(1,1), name= "G_res1_conv_2_2")(G_res1_zeroPad_2_2)
G_res1_sumImg_2 = Add(name ="add_6")([G_res1_convImg_2_2, G_res1_sumImg_1])
G_res1_reluImg_2_2 = ReLU(name= "G_res1_relu_2_2")(G_res1_sumImg_2)

G_res1_zeroPad_3_1 = ZeroPadding2D((2,2), name= "G_res1_zero_3_1")(G_res1_reluImg_2_2)
G_res1_convImg_3_1 = Convolution2D(64, (5, 5), strides=(1,1), name= "G_res1_conv_3_1")(G_res1_zeroPad_3_1)
G_res1_reluImg_3_1 = ReLU(name= "G_res1_relu_3_1")(G_res1_convImg_3_1)
G_res1_zeroPad_3_2 = ZeroPadding2D((2,2), name= "G_res1_zero_3_2")(G_res1_reluImg_3_1)
G_res1_convImg_3_2 = Convolution2D(64, (5, 5), strides=(1,1), name= "G_res1_conv_3_2")(G_res1_zeroPad_3_2)
G_res1_sumImg_3 = Add(name = "add_7")([G_res1_convImg_3_2, G_res1_sumImg_2])
G_res1_reluImg_3_2 = ReLU(name= "G_res1_relu_3_2")(G_res1_sumImg_3)

G_res1_zeroPad_4_1 = ZeroPadding2D((2,2), name= "G_res1_zero_4_1")(G_res1_reluImg_3_2)
G_res1_convImg_4_1 = Convolution2D(64, (5, 5), strides=(1,1), name= "G_res1_conv_4_1")(G_res1_zeroPad_4_1)
G_res1_reluImg_4_1 = ReLU(name= "G_res1_relu_4_1")(G_res1_convImg_4_1)
G_res1_zeroPad_4_2 = ZeroPadding2D((2,2), name= "G_res1_zero_4_2")(G_res1_reluImg_4_1)
G_res1_convImg_4_2 = Convolution2D(64, (5, 5), strides=(1,1), name= "G_res1_conv_4_2")(G_res1_zeroPad_4_2)
G_res1_sumImg_4 = Add(name = "add_8")([G_res1_convImg_4_2, G_res1_sumImg_3])
G_res1_reluImg_4_2 = ReLU(name= "G_res1_relu_4_2")(G_res1_sumImg_4)

##### yunus emre 
G_res1_zeroPad_5_1 = ZeroPadding2D((2,2), name= "G_res1_zero_5_1")(G_res1_reluImg_4_2)
G_res1_convImg_5_1 = Convolution2D(64, (5, 5), strides=(1,1), name= "G_res1_conv_5_1")(G_res1_zeroPad_5_1)
G_res1_zeroPad_5_2 = ZeroPadding2D((2,2), name= "G_res1_zero_5_2")(G_res1_convImg_5_1)
G_res1_convImg_5_2 = Convolution2D(64, (5, 5), strides=(1,1), name= "G_res1_conv_5_2")(G_res1_zeroPad_5_2)
G_res1_sumImg_5 = Add(name = "add_9")([G_res1_convImg_5_2,G_res1_sumImg_4])
G_res1_reluImg_5_2 = ReLU(name= "G_res1_relu_5_2")(G_res1_sumImg_5)

G_res1_zeroPad_4 = ZeroPadding2D((2,2), name= "G_res1_zero_4")(G_res1_reluImg_5_2)
G_convImg_4 = Convolution2D(64, (5, 5), strides=(1,1), name= "G_res1_conv_4")(G_res1_zeroPad_4)
G_reluImg_4 = ReLU(name= "G_res1_relu_4")(G_convImg_4)

G_res1_zeroPad_5 = ZeroPadding2D((2,2), name= "G_res1_zero_5")(G_reluImg_4)
G_convImg_5 = Convolution2D(64, (5, 5), strides=(1,1), name= "G_res1_conv_5")(G_res1_zeroPad_5)
G_reluImg_5 = ReLU(name= "G_res1_relu_5")(G_convImg_5)

G_res1_zeroPad_6 = ZeroPadding2D((2,2), name= "G_res1_zero_6")(G_reluImg_5)
G_convImg_6 = Convolution2D(3, (5, 5), strides=(1,1), name= "G_res1_conv_6")(G_res1_zeroPad_6)

### SCALE_2
de_deconv = Conv2DTranspose(3, (4, 4), strides=(2,2), dilation_rate=(1,1), name= "de_deconv")(G_convImg_6)
de_crop = Cropping2D(cropping=((1, 1), (1, 1)))(de_deconv)
G_concatImg_12 = Concatenate(name= "G_concat_2")([de_crop, inputImg])
G_concatImg_2 = Concatenate(name= "G_concat_3")([G_concatImg_12, G_softImg_1])

G_convImg2_1_zero_pad = ZeroPadding2D((5,5), name= "G_convImg2_1_zero_pad")(G_concatImg_2)
G_convImg2_1 = Convolution2D(64, (11, 11), strides=(1,1), name= "G_convImg2_1")(G_convImg2_1_zero_pad)
G_reluImg2_1 = ReLU(name= "G_reluImg2_1")(G_convImg2_1)

G_convImg2_2_zero_pad = ZeroPadding2D((2,2), name= "G_convImg2_2_zero_pad")(G_reluImg2_1)
G_convImg2_2 = Convolution2D(64, (5, 5), strides=(1,1), name= "G_convImg2_2")(G_convImg2_2_zero_pad)
G_reluImg2_2 = ReLU(name= "G_reluImg2_2")(G_convImg2_2)

G_convImg2_3_zero_pad = ZeroPadding2D((2,2), name= "G_convImg2_3_zero_pad")(G_reluImg2_2)
G_convImg2_3 = Convolution2D(64, (5, 5), strides=(1,1), name= "G_convImg2_3")(G_convImg2_3_zero_pad)
G_reluImg2_3 = ReLU(name= "G_reluImg2_3")(G_convImg2_3)

G_res2_zeroPad_1_1 = ZeroPadding2D((2,2), name= "G_res2_zero_1_1")(G_reluImg2_3)
G_res2_convImg_1_1 = Convolution2D(64, (5, 5), strides=(1,1), name= "G_res2_conv_1_1")(G_res2_zeroPad_1_1)
G_res2_reluImg_1_1 = ReLU(name= "G_res2_relu_1_1")(G_res2_convImg_1_1)
G_res2_zeroPad_1_2 = ZeroPadding2D((2,2), name= "G_res2_zero_1_2")(G_res2_reluImg_1_1)
G_res2_convImg_1_2 = Convolution2D(64, (5, 5), strides=(1,1), name= "G_res2_conv_1_2")(G_res2_zeroPad_1_2)
G_res2_sumImg_1 = Add(name = "add_10")([G_res2_convImg_1_2, G_reluImg2_3])
G_res2_reluImg_1_2 = ReLU(name= "G_res2_relu_1_2")(G_res2_sumImg_1)


G_res2_zeroPad_2_1 = ZeroPadding2D((2,2), name= "G_res2_zeroPad_2_1")(G_res2_reluImg_1_2)
G_res2_convImg_2_1 = Convolution2D(64, (5, 5), strides=(1,1), name= "G_res2_conv_2_1")(G_res2_zeroPad_2_1)
G_res2_reluImg_2_1 = ReLU(name= "G_res2_relu_2_1")(G_res2_convImg_2_1)
G_res2_zeroPad_2_2 = ZeroPadding2D((2,2), name= "G_res2_zero_2_2")(G_res2_reluImg_2_1)
G_res2_convImg_2_2 = Convolution2D(64, (5, 5), strides=(1,1), name= "G_res2_conv_2_2")(G_res2_zeroPad_2_2)
G_res2_sumImg_2 = Add(name = "add_11")([G_res2_convImg_2_2, G_res2_sumImg_1])
G_res2_reluImg_2_2 = ReLU(name= "G_res2_relu_2_2")(G_res2_sumImg_2)


G_res2_zeroPad_3_1 = ZeroPadding2D((2,2), name= "G_res2_zero_3_1")(G_res2_reluImg_2_2)
G_res2_convImg_3_1_deneme = Convolution2D(64, (5, 5), strides=(1,1), name= "G_res2_conv_3_1_deneme")(G_res2_zeroPad_3_1) ### !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!1
G_res2_reluImg_3_1 = ReLU(name= "G_res2_relu_3_1")(G_res2_convImg_3_1_deneme)
G_res2_zeroPad_3_2 = ZeroPadding2D((2,2), name= "G_res2_zero_3_2")(G_res2_reluImg_3_1)
G_res2_convImg_3_2 = Convolution2D(64, (5, 5), strides=(1,1), name= "G_res2_conv_3_2")(G_res2_zeroPad_3_2)
G_res2_sumImg_3 = Add(name = "add_12")([G_res2_convImg_3_2, G_res2_sumImg_2])
G_res2_reluImg_3_2 = ReLU(name= "G_res2_relu_3_2")(G_res2_sumImg_3)


G_res2_zeroPad_4_1 = ZeroPadding2D((2,2), name= "G_res2_zero_4_1")(G_res2_reluImg_3_2)
G_res2_convImg_4_1 = Convolution2D(64, (5, 5), strides=(1,1), name= "G_res2_conv_4_1")(G_res2_zeroPad_4_1)
G_res2_reluImg_4_1 = ReLU(name= "G_res2_relu_4_1")(G_res2_convImg_4_1)
G_res2_zeroPad_4_2 = ZeroPadding2D((2,2), name= "G_res2_zero_4_2")(G_res2_reluImg_4_1)
G_res2_convImg_4_2 = Convolution2D(64, (5, 5), strides=(1,1), name= "G_res2_conv_4_2")(G_res2_zeroPad_4_2)
G_res2_sumImg_4 = Add(name = "add_13")([G_res2_convImg_4_2, G_res2_sumImg_3])
G_res2_reluImg_4_2 = ReLU(name= "G_res2_relu_4_2")(G_res2_sumImg_4)


G_res2_zeroPad_5_1 = ZeroPadding2D((2,2), name= "G_res2_zero_5_1")(G_res2_reluImg_4_2)
G_res2_convImg_5_1 = Convolution2D(64, (5, 5), strides=(1,1), name= "G_res2_conv_5_1")(G_res2_zeroPad_5_1)
G_res2_reluImg_5_1 = ReLU(name= "G_res2_relu_5_1")(G_res2_convImg_5_1)
G_res2_zeroPad_5_2 = ZeroPadding2D((2,2), name= "G_res2_zero_5_2")(G_res2_reluImg_5_1)
G_res2_convImg_5_2 = Convolution2D(64, (5, 5), strides=(1,1), name= "G_res2_conv_5_2")(G_res2_zeroPad_5_2)
G_res2_sumImg_5 = Add(name = "add_14")([G_res2_convImg_5_2, G_res2_sumImg_4])
G_res2_reluImg_5_2 = ReLU(name= "G_res2_relu_5_2")(G_res2_sumImg_5)


G_2convImg_4_zero_pad = ZeroPadding2D((2,2), name= "G_2convImg_4_zero_pad")(G_res2_reluImg_5_2)
G_2convImg_4 = Convolution2D(64, (5, 5), strides=(1,1), name= "G_convImg4")(G_2convImg_4_zero_pad)
G_2reluImg_4 = ReLU(name= "G_reluImg4")(G_2convImg_4)

G_2convImg_5_zero_pad = ZeroPadding2D((2,2), name= "G_2convImg_5_zero_pad")(G_2reluImg_4)
G_2convImg_5 = Convolution2D(64, (5, 5), strides=(1,1), name= "G_convImg5")(G_2convImg_5_zero_pad)
G_2reluImg_5 = ReLU(name= "G_reluImg5")(G_2convImg_5)


G_convImg_6_zero_pad = ZeroPadding2D((2,2), name= "G_convImg_6_zero_pad")(G_2reluImg_5)
G_convImg_6_last = Convolution2D(3, (5, 5), strides=(1,1), name= "G_convImg_6_last")(G_convImg_6_zero_pad)

modelP = Model(inputs=[inputImg], outputs=de_convImg_parsing)
modelG = Model(inputs=[modelG_Input,inputImg0, inputImg ], outputs=G_convImg_6_last)

In [160]:
modelG.summary()

Model: "model_18"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_24 (InputLayer)           [(None, 128, 128, 11 0                                            
__________________________________________________________________________________________________
G_soft_1 (Softmax)              (None, 128, 128, 11) 0           input_24[0][0]                   
__________________________________________________________________________________________________
input_23 (InputLayer)           [(None, 64, 64, 3)]  0                                            
__________________________________________________________________________________________________
G_maxpool_1 (MaxPooling2D)      (None, 64, 64, 11)   0           G_soft_1[0][0]                   
___________________________________________________________________________________________

In [161]:
modelP.layers[3].weights
print(netP[0,4].bm.shape)

(256, 2)


In [162]:
str(modelP.layers[3])

w_layer_index = {}

conv_counter = 1
norm_counter = 1
tr_conv_counter = 1
conv_list = []
batch_list = []
tconv_list = []

for i in range(len(modelP.layers)):
    layer_name = str(modelP.layers[i])


    if "BatchNormalization" in layer_name:
        norm_counter += 1
        batch_list.append(i)
        
    if "Conv2DTranspose" in layer_name:
        tr_conv_counter += 1
        tconv_list.append(i)
    
    elif "Conv2D" in layer_name:
        conv_list.append(i)
        conv_counter += 1
        #print(modelP.layers[i].name)
conv_counter
# "normalization"
# convolution

13

In [163]:
counter = 0
model_index = 0
tconv_index = 0
batch_index = 0
for i in range(netP.shape[1]):
    field_names = netP[0][i].__dict__["_fieldnames"]
    a = i + 1
        
    for name in field_names:
        #if model_index != 28:
        if name == "w":
            print(i, model_index, modelP.layers[conv_list[model_index]].weights[0].shape,"-------", netP[0][i].w.shape)
            modelP.layers[conv_list[model_index]].set_weights([netP[0][i].w, netP[0][i].b.reshape(netP[0][i].b.shape[0], )])
            model_index += 1
            counter +=1

0 0 (3, 3, 3, 16) ------- (3, 3, 3, 16)
1 1 (3, 3, 16, 32) ------- (3, 3, 16, 32)
2 2 (5, 5, 32, 64) ------- (5, 5, 32, 64)
3 3 (3, 3, 64, 128) ------- (3, 3, 64, 128)
4 4 (3, 3, 128, 256) ------- (3, 3, 128, 256)
5 5 (3, 3, 256, 512) ------- (3, 3, 256, 512)
6 6 (3, 3, 512, 256) ------- (3, 3, 512, 256)
7 7 (3, 3, 256, 128) ------- (3, 3, 256, 128)
8 8 (3, 3, 128, 64) ------- (3, 3, 128, 64)
9 9 (3, 3, 64, 32) ------- (3, 3, 64, 32)
10 10 (3, 3, 32, 16) ------- (3, 3, 32, 16)
11 11 (3, 3, 16, 11) ------- (3, 3, 16, 11)


In [164]:
netP.shape[1]

12

# PARSING NETWORK

In [165]:
counter = 0
model_index = 0
tconv_index = 0
batch_index = 0

for i in range(netP.shape[1]):
    field_names = netP[0][i].__dict__["_fieldnames"]
    a = i + 1
        
    for name in field_names:
        #if model_index != 28:
        if name == "w":
            print(i, model_index, modelP.layers[conv_list[model_index]].weights[0].shape,"-------", netP[0][i].w.shape)
            modelP.layers[conv_list[model_index]].set_weights([netP[0][i].w, netP[0][i].b.reshape(netP[0][i].b.shape[0], )])
            model_index += 1
            counter +=1

        
        if name == "bw":
            print(i)
            if batch_index == len(batch_list):
                print("sona gelindi")
                break
            print(i, batch_index, modelP.layers[batch_list[batch_index]].weights[0].shape,"bbbbbbbbb", netP[0][i].bw.shape)
            bm1 = np.copy(net[0][i].bm[:,0].reshape(netP[0][i].bb.shape[0],))
            bm2 = np.copy(net[0][i].bm[:,1].reshape(netP[0][i].bb.shape[0],))

            modelP.layers[batch_list[batch_index]].set_weights([ netP[0][i].bw.reshape(netP[0,i].bw.shape[0],), net[0][i].bb.reshape(netP[0][i].bb.shape[0]), bm1, bm2])
            #print(i, batch_index, modelP.layers[batch_list[batch_index]].weights[0].shape,"-------", netP[0][i].w.shape)
            batch_index += 1
            counter +=1

        if name == "upw":
            if netP[0][i].upb.shape[0] != 1:
                print(str(a), name + " shape", net[0][i].upw.shape )
                modelP.layers[tconv_list[tconv_index]].set_weights([netP[0][i].upw, net[0][i].upb.reshape(netP[0][i].upb.shape[0])])
                
                tconv_index += 1
                counter +=1


0 0 (3, 3, 3, 16) ------- (3, 3, 3, 16)
0
0 0 (16,) bbbbbbbbb (16, 1)
1 1 (3, 3, 16, 32) ------- (3, 3, 16, 32)
1
1 1 (32,) bbbbbbbbb (32, 1)
2 2 (5, 5, 32, 64) ------- (5, 5, 32, 64)
2
2 2 (64,) bbbbbbbbb (64, 1)
3 3 (3, 3, 64, 128) ------- (3, 3, 64, 128)
3
3 3 (128,) bbbbbbbbb (128, 1)
4 4 (3, 3, 128, 256) ------- (3, 3, 128, 256)
4
4 4 (256,) bbbbbbbbb (256, 1)
5 5 (3, 3, 256, 512) ------- (3, 3, 256, 512)
5
5 5 (512,) bbbbbbbbb (512, 1)
6 6 (3, 3, 512, 256) ------- (3, 3, 512, 256)
6
6 6 (256,) bbbbbbbbb (256, 1)
7 upw shape (4, 4, 512, 512)
7 7 (3, 3, 256, 128) ------- (3, 3, 256, 128)
7
7 7 (128,) bbbbbbbbb (128, 1)
8 upw shape (4, 4, 256, 256)
8 8 (3, 3, 128, 64) ------- (3, 3, 128, 64)
8
8 8 (64,) bbbbbbbbb (64, 1)
9 upw shape (4, 4, 128, 128)
9 9 (3, 3, 64, 32) ------- (3, 3, 64, 32)
9
9 9 (32,) bbbbbbbbb (32, 1)
10 upw shape (4, 4, 64, 64)
10 10 (3, 3, 32, 16) ------- (3, 3, 32, 16)
10
10 10 (16,) bbbbbbbbb (16, 1)
11 upw shape (4, 4, 32, 32)
11 11 (3, 3, 16, 11) ------- (3,

In [166]:
import cv2 
modelParsing = Model(inputs=[inputImg], outputs=de_convImg_parsing)

## GENERATIVE NETWORK

In [167]:
str(modelG.layers[3])

w_layer_index = {}

conv_counter = 1
norm_counter = 1
tr_conv_counter = 1
conv_list = []
batch_list = []
tconv_list = []

for i in range(len(modelG.layers)):
    layer_name = str(modelG.layers[i])
    
    if "Conv2D" in layer_name or "Conv2DTranspose" in layer_name:
        conv_list.append(i)
        conv_counter += 1
        #print(modelG.layers[i].name)
# "normalization"
# convolution
conv_list

[6,
 9,
 12,
 15,
 18,
 22,
 25,
 29,
 32,
 36,
 39,
 43,
 45,
 49,
 52,
 55,
 56,
 62,
 65,
 68,
 71,
 74,
 78,
 81,
 85,
 88,
 92,
 95,
 99,
 102,
 106,
 109,
 112]

In [168]:
counter = 0
model_index = 0
tconv_index = 0
batch_index = 0

for i in range(netG.shape[1]):
    field_names = netG[0][i].__dict__["_fieldnames"]
    a = i + 1
    for name in field_names:
        try: 
            print("++++++++++++++++++++", name)

            #if model_index != 28:
            if name == "w":
                #print("shapeeeeee", np.array(modelG.layers[conv_list[model_index]].get_weights()).shape)
                print("conv",i, model_index, modelG.layers[conv_list[model_index]].weights[0].shape,"-------", netG[0][i].w.shape)

                modelG.layers[conv_list[model_index]].set_weights([netG[0][i].w, netG[0][i].b.reshape(netG[0][i].b.shape[0], )])
                if np.all(modelG.layers[conv_list[model_index]].get_weights()[1] == netG[0][i].b.reshape(netG[0][i].b.shape[0], )): 
                    pass
                else:
                    print("hata vaaaaaar")
                    break 
                model_index += 1
                counter +=1

            
            if name == "bw":
                print(i)
                if batch_index == len(batch_list):
                    print("sona gelindi")
                    break
                print("batch", i, batch_index, modelG.layers[batch_list[batch_index]].weights[0].shape,"bbbbbbbbb", netG[0][i].bw.shape)
                bm1 = np.copy(net[0][i].bm[:,0].reshape(netG[0][i].bb.shape[0],))
                bm2 = np.copy(net[0][i].bm[:,1].reshape(netG[0][i].bb.shape[0],))

                modelG.layers[batch_list[batch_index]].set_weights([ netG[0][i].bw.reshape(netG[0,i].bw.shape[0],), netG[0][i].bb.reshape(netG[0][i].bb.shape[0]), bm1, bm2])
                #print(i, batch_index, modelG.layers[batch_list[batch_index]].weights[0].shape,"-------", netG[0][i].w.shape)
                batch_index += 1
                counter +=1

            if name == "upw":
                if netG[0][i].upb.shape[0] != 1:
                    print("transpose",str(a), name + " shape", net[0][i].upw.shape )
                    modelG.layers[tconv_list[tconv_index]].set_weights([netG[0][i].upw, net[0][i].upb.reshape(netG[0][i].upb.shape[0])])
                    
                    tconv_index += 1
                    counter +=1
        except Exception as e:
            print("------------------------------", e)
            model_index += 1

++++++++++++++++++++ w
conv 0 0 (11, 11, 14, 64) ------- (11, 11, 14, 64)
++++++++++++++++++++ b
++++++++++++++++++++ w
conv 1 1 (5, 5, 64, 64) ------- (5, 5, 64, 64)
++++++++++++++++++++ b
++++++++++++++++++++ w
conv 2 2 (5, 5, 64, 64) ------- (5, 5, 64, 64)
++++++++++++++++++++ b
++++++++++++++++++++ w
conv 3 3 (5, 5, 64, 64) ------- (5, 5, 64, 64)
++++++++++++++++++++ b
++++++++++++++++++++ w
conv 4 4 (5, 5, 64, 64) ------- (5, 5, 64, 64)
++++++++++++++++++++ b
++++++++++++++++++++ w
conv 5 5 (5, 5, 64, 64) ------- (5, 5, 64, 64)
++++++++++++++++++++ b
++++++++++++++++++++ w
conv 6 6 (5, 5, 64, 64) ------- (5, 5, 64, 64)
++++++++++++++++++++ b
++++++++++++++++++++ w
conv 7 7 (5, 5, 64, 64) ------- (5, 5, 64, 64)
++++++++++++++++++++ b
++++++++++++++++++++ w
conv 8 8 (5, 5, 64, 64) ------- (5, 5, 64, 64)
++++++++++++++++++++ b
++++++++++++++++++++ w
conv 9 9 (5, 5, 64, 64) ------- (5, 5, 64, 64)
++++++++++++++++++++ b
++++++++++++++++++++ w
conv 10 10 (5, 5, 64, 64) ------- (5, 5, 64

In [176]:
image = cv2.imread("example.png")
#image = cv2.resize(image, (128,128), interpolation=cv2.INTER_CUBIC)
#cv2.imwrite("small.png", image)
image = cv2.cvtColor(image ,cv2.COLOR_BGR2RGB)
image=image.astype(np.float64)/255.0
image_resized = np.reshape(image, (1,128,128,3)) 
print(image_resized.shape)
print(image.shape)
#save model and structure alone
#modelParsing.save("model.hdf5")
#modelParsing.compile(optimizer='adam')
result = modelP.predict(image_resized)

img_new = cv2.resize(image, (64,64))
img_small = np.reshape(img_new, (1,64,64,3))
res = modelG.predict([result, img_small, image_resized] )
res = res.reshape((128,128,3))

res = np.clip(res, 0.0, 1.0) * 255.0
res = res.astype(np.uint8)
res = cv2.cvtColor(res ,cv2.COLOR_RGB2BGR)
#res_rgb = cv2.cvtColor(res, cv2.COLOR_BGR2RGB)
cv2.imwrite("res2.png", res)

(1, 128, 128, 3)
(128, 128, 3)


True

In [178]:
img=cv2.imread('example.png')
img=cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
img=img.astype(np.float)/255.0
print(img.shape)
halfImg=cv2.resize(img,None,fx=0.5,fy=0.5,interpolation=cv2.INTER_CUBIC)
print(halfImg.shape)
img=img.reshape((1,128,128,3))
halfImg=halfImg.reshape((1,64,64,3))
result = modelP.predict(img)
result =result.reshape((128,128,11))
for i in range(result.shape[2]):
    cv2.imwrite("semantic"+ str(i) +".png", result[:,:,i])

#res = modelG.predict([result, halfImg, img] )
#res = np.clip(res, 0.0, 1.0) * 255.0
#res = res.astype(np.uint8)
#res=cv2.cvtColor(res,cv2.COLOR_RGB2BGR)
#cv2.imwrite("res.png", res)



<ipython-input-178-498a37654cfa>:3: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  img=img.astype(np.float)/255.0


(128, 128, 3)
(64, 64, 3)


In [72]:
np.all(modelG.weights[1][:] == netG[0][0].b.reshape(netG[0,0].b.shape[0], ))

True

In [70]:
netG[0][0].b.reshape(netG[0,0].b.shape[0], )

array([-0.02284474,  0.0011319 , -0.0049351 , -0.02105369, -0.00611973,
       -0.01545282, -0.0100606 , -0.03104383,  0.00301641, -0.00232907,
       -0.04532468,  0.07328062, -0.0132912 , -0.02339904, -0.02200021,
       -0.02699594,  0.01490882, -0.01337147, -0.03548499, -0.01633993,
        0.00879346,  0.00675953,  0.03981332, -0.02652108, -0.02779853,
       -0.02611826, -0.02641862, -0.02430061, -0.01131134,  0.02057475,
       -0.0113089 , -0.02544295,  0.00136867,  0.07211313, -0.01462926,
        0.02267528,  0.05124017,  0.00173653,  0.01872467,  0.05447832,
       -0.01879578, -0.00442702, -0.01998405, -0.01044124, -0.0046479 ,
       -0.01771142,  0.0093828 ,  0.01550732,  0.07619891, -0.02462062,
       -0.00360243,  0.01852007, -0.02976297, -0.01096003, -0.03119661,
        0.07503627,  0.00940416, -0.01403641, -0.00969108, -0.03153072,
        0.00429602,  0.00106897, -0.02543839, -0.01865718], dtype=float32)

In [56]:
modelG.weights[0][:].shape

TensorShape([11, 11, 14, 64])